In [53]:
from draft_kings import Client
from draft_kings.data import Sport
import pprint
import pandas as pd
import requests
from typing import List
import time

In [2]:
contest_id = 182136738

In [3]:
client = Client()

# Get current NFL contests
contests_details = client.contests(sport=Sport.NFL)

In [13]:
client.contests(sport=Sport.NFL).contests[

ContestDetails(contest_id=182136738, draft_group_id=133903, entries_details=EntriesDetails(fee=555.0, maximum=6006, total=826), fantasy_player_points=277.0, is_double_up=False, is_fifty_fifty=False, is_guaranteed=True, is_head_to_head=False, is_starred=True, name='NFL $3M Fantasy Football Millionaire [$1M to 1st]', payout=3000000.0, sport=<Sport.NFL: 'NFL'>, starts_at=datetime.datetime(2025, 9, 21, 17, 0, tzinfo=datetime.timezone.utc))

In [6]:
# contest_list = []

# for i in range(len(client.contests(sport=Sport.NFL).contests)):
#     dk_id = client.contests(sport=Sport.NFL).contests[i].contest_id
#     contest_list.append(dk_id)

In [26]:
filtered_contests = [contest for contest in contests_details.contests 
               if contest.is_guaranteed 
               and contest.draft_group_id == 133903
               and contest.is_double_up == False
               and contest.is_fifty_fifty == False]

In [27]:
contest_ids = [contest.contest_id for contest in contests_details.contests 
               if contest.is_guaranteed 
               and contest.draft_group_id == 133903
               and contest.is_double_up == False
               and contest.is_fifty_fifty == False]

In [31]:
# Convert to DataFrame
df = pd.DataFrame([contest.__dict__ for contest in filtered_contests])

In [33]:
contest_data = []
for contest in filtered_contests:
    contest_dict = {
        'contest_id': contest.contest_id,
        'draft_group_id': contest.draft_group_id,
        'fee': contest.entries_details.fee,
        'maximum': contest.entries_details.maximum,
        'total': contest.entries_details.total,
        'fantasy_player_points': contest.fantasy_player_points,
        'is_double_up': contest.is_double_up,
        'is_fifty_fifty': contest.is_fifty_fifty,
        'is_guaranteed': contest.is_guaranteed,
        'is_head_to_head': contest.is_head_to_head,
        'is_starred': contest.is_starred,
        'name': contest.name,
        'payout': contest.payout,
        'sport': contest.sport.value if hasattr(contest.sport, 'value') else str(contest.sport),
        'starts_at': contest.starts_at
    }
    contest_data.append(contest_dict)

# Convert to DataFrame
df = pd.DataFrame(contest_data)

In [34]:
df

,contest_id,draft_group_id,fee,maximum,total,fantasy_player_points,is_double_up,is_fifty_fifty,is_guaranteed,is_head_to_head,is_starred,name,payout,sport,starts_at
0,182136735,133903,20.0,191176,23978,10.0,False,False,True,False,True,NFL $3.25M Fantasy Football Millionaire [$1M t...,3250000.0,NFL,2025-09-21 17:00:00+00:00
1,182136738,133903,555.0,6006,825,277.0,False,False,True,False,True,NFL $3M Fantasy Football Millionaire [$1M to 1st],3000000.0,NFL,2025-09-21 17:00:00+00:00
2,182136737,133903,3.0,352313,53611,1.0,False,False,True,False,True,NFL $888K Play-Action [20 Entry Max],888888.0,NFL,2025-09-21 17:00:00+00:00
3,182136740,133903,3180.0,300,38,1590.0,False,False,True,False,True,NFL $900K Luxury Box [$200K to 1st],900000.0,NFL,2025-09-21 17:00:00+00:00
4,182136739,133903,150.0,3703,780,75.0,False,False,True,False,True,"NFL $500K Power Sweep [$100K to 1st, 3 Entry Max]",500000.0,NFL,2025-09-21 17:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,182152186,133903,1.0,31,16,0.0,False,False,True,False,False,NFL $1 Triple Up [Top 9 Win $3],27.0,NFL,2025-09-21 17:00:00+00:00
715,182152185,133903,1.0,31,13,0.0,False,False,True,False,False,NFL $1 Triple Up [Top 9 Win $3],27.0,NFL,2025-09-21 17:00:00+00:00
716,182152184,133903,1.0,31,10,0.0,False,False,True,False,False,NFL $1 Triple Up [Top 9 Win $3],27.0,NFL,2025-09-21 17:00:00+00:00
717,182152183,133903,1.0,31,16,0.0,False,False,True,False,False,NFL $1 Triple Up [Top 9 Win $3],27.0,NFL,2025-09-21 17:00:00+00:00


In [46]:
url = f'https://api.draftkings.com/contests/v1/contests/{i}?format=json'

In [36]:
url = 'https://api.draftkings.com/contests/v1/contests/182136735?format=json'
r = requests.get(url)

In [44]:
api_pull = r.json()['contestDetail']

# Extract basic contest information
contest_key = api_pull['contestKey']
entry_fee = api_pull['entryFee']
total_payouts = api_pull['totalPayouts']
maximum_entries = api_pull['maximumEntries']
maximum_entries_per_user = api_pull['maximumEntriesPerUser']
name = api_pull['name']

# Extract payout information
payout_summary = api_pull['payoutSummary']

# Find the lowest minPosition (should be 1 for first place)
lowest_min_position = min(payout['minPosition'] for payout in payout_summary)
highest_payout_value = None

for payout in payout_summary:
    if payout['minPosition'] == lowest_min_position:
        highest_payout_value = payout['payoutDescriptions'][0]['value']
        break

# Find the highest maxPosition (last place that pays out)
highest_max_position = max(payout['maxPosition'] for payout in payout_summary)
lowest_payout_value = None

for payout in payout_summary:
    if payout['maxPosition'] == highest_max_position:
        lowest_payout_value = payout['payoutDescriptions'][0]['value']
        break

# Store the place values
highest_place_paid = lowest_min_position  # Best place (1st)
lowest_place_paid = highest_max_position  # Worst place that still pays

# Create the DataFrame
df_data = {
    'contestKey': [contest_key],
    'entryFee': [entry_fee],
    'totalPayouts': [total_payouts],
    'maximumEntries': [maximum_entries],
    'maximumEntriesPerUser': [maximum_entries_per_user],
    'name': [name],
    'highest_place_paid': [highest_place_paid],      # Best place (1st)
    'lowest_place_paid': [lowest_place_paid],        # Worst place that still pays (43,175th)
    'highest_payout_value': [highest_payout_value],  # Value for 1st place
    'lowest_payout_value': [lowest_payout_value]     # Value for last place that pays
}

df = pd.DataFrame(df_data)

In [45]:
df

,contestKey,entryFee,totalPayouts,maximumEntries,maximumEntriesPerUser,name,highest_place_paid,lowest_place_paid,highest_payout_value,lowest_payout_value
0,182136735,20,3250000.0,191176,150,NFL $3.25M Fantasy Football Millionaire [$1M t...,1,43175,1000000.0,30.0


In [55]:
def process_contest_detail(contest_detail):
    """Process a single contest detail and return a dictionary with the extracted data"""
    
    # Extract basic contest information
    contest_key = contest_detail['contestKey']
    entry_fee = contest_detail['entryFee']
    total_payouts = contest_detail['totalPayouts']
    maximum_entries = contest_detail['maximumEntries']
    maximum_entries_per_user = contest_detail['maximumEntriesPerUser']
    name = contest_detail['name']

    # Extract payout information
    payout_summary = contest_detail['payoutSummary']

    # Find the lowest minPosition (should be 1 for first place)
    lowest_min_position = min(payout['minPosition'] for payout in payout_summary)
    highest_payout_value = None

    for payout in payout_summary:
        if payout['minPosition'] == lowest_min_position:
            highest_payout_value = payout['payoutDescriptions'][0]['value']
            break

    # Find the highest maxPosition (last place that pays out)
    highest_max_position = max(payout['maxPosition'] for payout in payout_summary)
    lowest_payout_value = None

    for payout in payout_summary:
        if payout['maxPosition'] == highest_max_position:
            lowest_payout_value = payout['payoutDescriptions'][0]['value']
            break

    # Store the place values
    highest_place_paid = lowest_min_position  # Best place (1st)
    lowest_place_paid = highest_max_position  # Worst place that still pays

    return {
        'contestKey': contest_key,
        'entryFee': entry_fee,
        'totalPayouts': total_payouts,
        'maximumEntries': maximum_entries,
        'maximumEntriesPerUser': maximum_entries_per_user,
        'name': name,
        'highest_place_paid': highest_place_paid,
        'lowest_place_paid': lowest_place_paid,
        'highest_payout_value': highest_payout_value,
        'lowest_payout_value': lowest_payout_value
    }

def fetch_contest_data_from_api(contest_ids: List, delay: float = 0.1):
    """
    Fetch contest data from DraftKings API for multiple contest IDs
    
    Args:
        contest_ids: List of contest IDs to fetch
        delay: Delay between requests in seconds (to be respectful to the API)
    
    Returns:
        pandas.DataFrame with contest data
    """
    contests_data = []
    failed_requests = []
    
    print(f"Fetching data for {len(contest_ids)} contests...")
    
    for i, contest_id in enumerate(contest_ids):
        url = f'https://api.draftkings.com/contests/v1/contests/{contest_id}?format=json'
        
        try:
            # print(f"Fetching contest {i+1}/{len(contest_ids)}: {contest_id}")
            
            # Make the API request
            response = requests.get(url)
            response.raise_for_status()  # Raises an exception for bad status codes
            
            # Parse JSON response
            data = response.json()
            
            # Check if contestDetail exists in the response
            if 'contestDetail' in data:
                contest_detail = data['contestDetail']
                contests_data.append(process_contest_detail(contest_detail))
            else:
                print(f"Warning: No contestDetail found for contest {contest_id}")
                failed_requests.append(contest_id)
            
            # Be respectful to the API - add delay between requests
            time.sleep(delay)
            
        except requests.exceptions.RequestException as e:
            print(f"Error fetching contest {contest_id}: {e}")
            failed_requests.append(contest_id)
            continue
        except KeyError as e:
            print(f"Error parsing data for contest {contest_id}: Missing key {e}")
            failed_requests.append(contest_id)
            continue
        except Exception as e:
            print(f"Unexpected error for contest {contest_id}: {e}")
            failed_requests.append(contest_id)
            continue
    
    print(f"\nSuccessfully fetched {len(contests_data)} contests")
    if failed_requests:
        print(f"Failed to fetch {len(failed_requests)} contests: {failed_requests}")
    
    return pd.DataFrame(contests_data)

In [56]:
df = fetch_contest_data_from_api(contest_ids)

Fetching data for 719 contests...
Fetching contest 1/719: 182136735
Fetching contest 2/719: 182136738
Fetching contest 3/719: 182136737
Fetching contest 4/719: 182136740
Fetching contest 5/719: 182136739
Fetching contest 6/719: 182153731
Fetching contest 7/719: 182153768
Fetching contest 8/719: 182153754
Fetching contest 9/719: 182153776
Fetching contest 10/719: 182153784
Fetching contest 11/719: 182153785
Fetching contest 12/719: 182153786
Fetching contest 13/719: 182153782
Fetching contest 14/719: 182153735
Fetching contest 15/719: 182153736
Fetching contest 16/719: 182153745
Fetching contest 17/719: 182153729
Fetching contest 18/719: 182153744
Fetching contest 19/719: 182225630
Fetching contest 20/719: 182153780
Fetching contest 21/719: 182153734
Fetching contest 22/719: 182153779
Fetching contest 23/719: 182153738
Fetching contest 24/719: 182153739
Fetching contest 25/719: 182163534
Fetching contest 26/719: 182163529
Fetching contest 27/719: 182163533
Fetching contest 28/719: 18216

In [58]:
def create_payout_breakdown_df(contest_ids: List, delay: float = 0.1):
    """
    Create a DataFrame with individual records for each position and payout amount
    
    Args:
        contest_ids: List of contest IDs to fetch
        delay: Delay between requests in seconds
    
    Returns:
        pandas.DataFrame with columns: contestKey, position, payout_value
    """
    payout_records = []
    failed_requests = []
    
    print(f"Fetching payout data for {len(contest_ids)} contests...")
    
    for i, contest_id in enumerate(contest_ids):
        url = f'https://api.draftkings.com/contests/v1/contests/{contest_id}?format=json'
        
        try:
            # print(f"Processing contest {i+1}/{len(contest_ids)}: {contest_id}")
            
            # Make the API request
            response = requests.get(url)
            response.raise_for_status()
            
            # Parse JSON response
            data = response.json()
            
            if 'contestDetail' in data:
                contest_detail = data['contestDetail']
                contest_key = contest_detail['contestKey']
                payout_summary = contest_detail['payoutSummary']
                
                # Process each payout tier
                for payout_tier in payout_summary:
                    min_pos = payout_tier['minPosition']
                    max_pos = payout_tier['maxPosition']
                    payout_value = payout_tier['payoutDescriptions'][0]['value']
                    
                    # Create a record for each individual position
                    for position in range(min_pos, max_pos + 1):
                        payout_records.append({
                            'contestKey': contest_key,
                            'position': position,
                            'payout_value': payout_value
                        })
                
                # print(f"  -> Added {sum(payout['maxPosition'] - payout['minPosition'] + 1 for payout in payout_summary)} position records")
            
            else:
                print(f"Warning: No contestDetail found for contest {contest_id}")
                failed_requests.append(contest_id)
            
            # Be respectful to the API
            time.sleep(delay)
            
        except requests.exceptions.RequestException as e:
            print(f"Error fetching contest {contest_id}: {e}")
            failed_requests.append(contest_id)
            continue
        except Exception as e:
            print(f"Unexpected error for contest {contest_id}: {e}")
            failed_requests.append(contest_id)
            continue
    
    print(f"\nSuccessfully processed {len(set(record['contestKey'] for record in payout_records))} contests")
    print(f"Total payout position records: {len(payout_records)}")
    
    if failed_requests:
        print(f"Failed to fetch {len(failed_requests)} contests: {failed_requests}")
    
    return pd.DataFrame(payout_records)

def create_payout_breakdown_from_existing_data(data):
    """
    Create payout breakdown from already fetched JSON data (single contest)
    
    Args:
        data: JSON data from API response
    
    Returns:
        pandas.DataFrame with position-level payout data
    """
    payout_records = []
    
    if 'contestDetail' in data:
        contest_detail = data['contestDetail']
        contest_key = contest_detail['contestKey']
        payout_summary = contest_detail['payoutSummary']
        
        # Process each payout tier
        for payout_tier in payout_summary:
            min_pos = payout_tier['minPosition']
            max_pos = payout_tier['maxPosition']
            payout_value = payout_tier['payoutDescriptions'][0]['value']
            
            # Create a record for each individual position
            for position in range(min_pos, max_pos + 1):
                payout_records.append({
                    'contestKey': contest_key,
                    'position': position,
                    'payout_value': payout_value
                })
    
    return pd.DataFrame(payout_records)

In [59]:
payout_df = create_payout_breakdown_df(contest_ids)

Fetching payout data for 719 contests...
  -> Added 43175 position records
  -> Added 1255 position records
  -> Added 85350 position records
  -> Added 61 position records
  -> Added 835 position records
  -> Added 306 position records
  -> Added 173 position records
  -> Added 20080 position records
  -> Added 67 position records
  -> Added 4135 position records
  -> Added 1524 position records
  -> Added 563 position records
  -> Added 999 position records
  -> Added 514 position records
  -> Added 258 position records
  -> Added 11 position records
  -> Added 7 position records
  -> Added 19 position records
  -> Added 1245 position records
  -> Added 367 position records
  -> Added 120 position records
  -> Added 71 position records
  -> Added 5805 position records
  -> Added 544 position records
  -> Added 4 position records
  -> Added 221 position records
  -> Added 4 position records
  -> Added 13 position records
  -> Added 46973 position records
  -> Added 75700 position reco

In [60]:
payout_df

,contestKey,position,payout_value
0,182136735,1,1000000.0
1,182136735,2,100000.0
2,182136735,3,50000.0
3,182136735,4,30000.0
4,182136735,5,20000.0
...,...,...,...
410530,182152183,5,3.0
410531,182152183,6,3.0
410532,182152183,7,3.0
410533,182152183,8,3.0
